<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>GHRSST24 2023</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://data.marine.copernicus.eu/register" target="_blank">A Copernicus Marine Service (CMEMS) account</a>** to enable you to download the data needed for this notebook.

There are no other prerequisite notebooks for this module, but you may wish to consider the following related notebooks:

- **<a href="https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/applications/ocean-case-studies/-/blob/main/Case_studies/Ocean_phenomena/Basin_scale_variability/Atlantic_Med_SST_anomalies_MHW_2023/Atlantic_Med_SST_anomalies_2023.ipynb" target="_blank">Sea surface temperature anomalies in the Northern Atlantic and Mediterranean Sea in 2023
</a>**
- **<a href="https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/applications/ocean-case-studies/-/blob/main/Case_studies/Ocean_phenomena/Basin_scale_variability/Atlantic_Med_SST_anomalies_MHW_2023/Med_MHW_2023.ipynb" target="_blank">Marine heatwaves in the Mediterranean Sea in 2023
</a>**
- **<a href="https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/sensors/learn-slstr" target="_blank">Learn SLSTR</a>**

</div>
<hr>

# Quick plotting GHRSST-format SST data

### Data used

| Product Description | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------:|:-----------------:|
| Global OSTIA SST (Reprocessed) | EO:MO:DAT:SST_GLO_SST_L4_REP_OBSERVATIONS_010_011 | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3ASST_GLO_SST_L4_REP_OBSERVATIONS_010_011" target="_blank">link</a> |
| Global OSTIA SST (Near real-time) | EO:MO:DAT:SST_GLO_SST_L4_NRT_OBSERVATIONS_010_001 | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AMO%3ADAT%3ASST_GLO_SST_L4_NRT_OBSERVATIONS_010_001" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* how to download data from the Copernicus Marine Service Data Store using OpenDap
* how to make a variety of simple plots with sea surface tmperature data
  
### Outline

...

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Preparating to run this script](#section1)
 1. [Downloading data](#section2)
 1. [Spatial plotting](#section3)
 1. [Calculating and adding fronts](#section4)

<hr>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
import os                                             # a library that allows us access to basic operating system commands
import datetime                                       # a library that allows us to work with dates and times
import numpy as np                                    # a library that lets us work with arrays; we import this with a new name "np"
import glob                                           # a package that helps with file searching
import xarray as xr                                   # a powerful library that helps us work efficiently with multi-dimensional arrays
import IPython                                        # a library that helps us display video and HTML content
import json                                           # a library that helps us make JSON format files
from pydap.client import open_url                     # a library that helps us access OpenDAP data
from pydap.cas.get_cookies import setup_session       # a library that helps us access OpenDAP data
import bokeh.plotting as bk                           # a library that supports plotting
from bokeh.palettes import *                          # as above
from bokeh.models import ColorBar, LinearColorMapper  # as above
import warnings                                       # a library the helps us to manage warnings

# turn off warnings
warnings.filterwarnings("ignore")

Plotting with Bokeh....

In [2]:
# initiate "bokeh" plot
bk.output_notebook()

Loading BokehJS ...

Fronts package....

In [3]:
if os.path.exists(os.path.join(os.getcwd(),"JUNO")):
    print("Package exists")
else:
    print("Fetching package")
    !git clone https://github.com/CoLAB-ATLANTIC/JUNO.git

from JUNO.src import CayulaCornillon_xarray

Fetching package
Cloning into 'JUNO'...
remote: Enumerating objects: 1800, done.
remote: Counting objects: 100% (309/309), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 1800 (delta 196), reused 275 (delta 172), pack-reused 1491
Receiving objects: 100% (1800/1800), 178.36 MiB | 12.62 MiB/s, done.
Resolving deltas: 100% (1238/1238), done.


Next we will create a download directory to store the products we will download in this notebook.

In [4]:
# Create a download directory for our SLSTR products
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-warning" role="alert">

## <a id='section0'></a>0. Defining functions
[Back to top](#TOC-TOP)

</div>

In [5]:
def fetch_cmems_opendap(dataset, authentication_url, times, region, output_template, credentials_file):
    '''
    This function prodives an OpenDAP connection to the CMEMS Data Store
    '''
    data_store = copernicus_datastore(authentication_url, dataset, credentials_file)
    DS = xr.open_dataset(data_store)
   
    files_list = []
    for time in times:
        print(f"Extracting data for {str(time)}")
        subset = DS.analysed_sst.sel(lon=slice(region[0], region[2]), lat=slice(region[1], region[3])).sel(time=time)
        output_file = os.path.join(download_dir, f"{output_template}{time.split('T')[0]}.nc")
        files_list.append(output_file)
        subset.to_netcdf(output_file)
    
    DS.close()
    data_store.close()

    return files_list

In [6]:
def copernicus_datastore(authentication_url, dataset, credentials_file):
    '''
    This function authenticates the CMEMS Data Store
    '''
    username, password = read_credentials_file(authentication_url, credentials_file, Oauth2=False)
    session = setup_session('https://cmems-cas.cls.fr/cas/login', username, password)
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])    
    data_store = xr.backends.PydapDataStore(open_url(f'{authentication_url}/thredds/dodsC/{dataset}',
                                                     session=session, user_charset='utf-8'))
    return data_store

In [7]:
def read_credentials_file(authentication_url, credentials_file=os.path.join(os.path.expanduser("~"),'.eumdac_credentials'), Oauth2=True):
    '''
    This function reads authentication details from the CMEMS or EUMDAC credantials files that are stored locally. 
    By default these are stored in the home directory as the .cmems_opendap and .eumdac_credentials files with the following structures:
    
    .eumdac_credentials:
    {
    "consumer_key": "<YOUR CONSUMER KEY>",
    "consumer_secret": "<YOUR CONSUMER SECRET>"
    }

    .cmems_opendap:
    {
    "https://my.cmems-du.eu": ["<YOUR USERNAME>", "<YOUR PASSWORD>"],
    "https://nrt.cmems-du.eu": ["<YOUR USERNAME>", "<YOUR PASSWORD>"]
    }

    Note that the files should have no extension (e.g. do not add .txt). You should replace the contents of the <> with the relevant
    details you obtain from EUMETSAT or CMEMS
    '''
    with open(credentials_file) as json_file:
        credentials = json.load(json_file)
        if Oauth2:
            key, secret = credentials['consumer_key'], credentials['consumer_secret']
            return key, secret
        else:
            username, password = credentials[authentication_url]
            return username, password

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Preparing to run this script
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Downloading data
[Back to top](#TOC_TOP)

</div>

### The CMEMS Data Store

In [8]:
# Data downloading options
download_cmems_data = True

In [9]:
# Credentials file
cmems_credentials_file = os.path.join(os.path.expanduser("~"),'.cmems_opendap')

In [10]:
# Multi-year reprocessing
authentication_url_my = 'https://my.cmems-du.eu'
dataset_my = 'METOFFICE-GLO-SST-L4-REP-OBS-SST'
format_my = 'CLIM_OSTIA_SST_'

# Near real-time
authentication_url_nrt = 'https://nrt.cmems-du.eu'
dataset_nrt = 'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2'
format_nrt = 'OPER_OSTIA_SST_'

In [11]:
# ROI
acquisition_ROI = [65.0, 0.0, 90.0, 25.0] # W, S, E, N

# climatology
start_year = 2000
end_year = 2019
month = 9
day = 24
clim_times = [f"{ii}-{str(month).zfill(2)}-{str(day).zfill(2)}T12:00:00.000000000" for ii in range(start_year, end_year+1)]

# operational
nrt_year = 2023
nrt_time = [f"{nrt_year}-{str(month).zfill(2)}-{str(day).zfill(2)}T12:00:00.000000000"]
formatted_month = datetime.datetime(nrt_year, month, day).strftime('%B')

In [12]:
if download_cmems_data:
    SST_files = glob.glob(os.path.join(download_dir, "*"))
    for f in SST_files:
        os.remove(f)

    clim_files = fetch_cmems_opendap(dataset_my, authentication_url_my, clim_times, acquisition_ROI, format_my, cmems_credentials_file)
    SST_file = fetch_cmems_opendap(dataset_nrt, authentication_url_nrt, nrt_time, acquisition_ROI, format_nrt, cmems_credentials_file)
else:
    clim_files = glob.glob(os.path.join(download_dir, format_my + "*"))
    SST_file = glob.glob(os.path.join(download_dir, format_nrt + "*"))

Extracting data for 2000-09-24T12:00:00.000000000
Extracting data for 2001-09-24T12:00:00.000000000
Extracting data for 2002-09-24T12:00:00.000000000
Extracting data for 2003-09-24T12:00:00.000000000
Extracting data for 2004-09-24T12:00:00.000000000
Extracting data for 2005-09-24T12:00:00.000000000
Extracting data for 2006-09-24T12:00:00.000000000
Extracting data for 2007-09-24T12:00:00.000000000
Extracting data for 2008-09-24T12:00:00.000000000
Extracting data for 2009-09-24T12:00:00.000000000
Extracting data for 2010-09-24T12:00:00.000000000
Extracting data for 2011-09-24T12:00:00.000000000
Extracting data for 2012-09-24T12:00:00.000000000
Extracting data for 2013-09-24T12:00:00.000000000
Extracting data for 2014-09-24T12:00:00.000000000
Extracting data for 2015-09-24T12:00:00.000000000
Extracting data for 2016-09-24T12:00:00.000000000
Extracting data for 2017-09-24T12:00:00.000000000
Extracting data for 2018-09-24T12:00:00.000000000
Extracting data for 2019-09-24T12:00:00.000000000


<div class="alert alert-info" role="alert">

## 3. <a id='section3'></a>Spatial plotting
[Back to top](#TOC_TOP)
    
</div>

In [13]:
SST_clim_data = xr.open_mfdataset(clim_files, combine='nested', concat_dim="time")
SST_clim = np.array(SST_clim_data["analysed_sst"].mean(dim='time'))
SST_data = xr.open_mfdataset(SST_file, combine='nested', concat_dim="time")
SST_oper = np.array(np.squeeze(SST_data["analysed_sst"]))
SST_anomaly = SST_oper - SST_clim

In [14]:
lonmin, lonmax = np.nanmin(SST_data["lon"]), np.nanmax(SST_data["lon"])
latmin, latmax = np.nanmin(SST_data["lat"]), np.nanmax(SST_data["lat"])

Plot the most recent data slice

In [15]:
color_mapper = LinearColorMapper(interp_palette(RdYlBu11, 256),
                                 low=np.nanmin(SST_oper),
                                 high=np.nanmax(SST_oper))
# plot
f1 = bk.figure(width=800, height=720, x_range=[lonmin, lonmax], y_range=[latmin, latmax])
f1.image(image=[SST_oper], x=[lonmin], y=[latmin], dw=[lonmax-lonmin], dh=[latmax-latmin], color_mapper=color_mapper)

clabel = f"OSTIA level-4 foundation temperature ({nrt_year}) for {day} {formatted_month}"
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0), title=clabel)
f1.add_layout(color_bar, 'right')

bk.show(f1)

Plot the anomaly vs the climatology

In [16]:
color_mapper = LinearColorMapper(interp_palette(RdBu11, 256),
                                 low=np.nanmin(SST_anomaly),
                                 high=np.nanmax(SST_anomaly))
# plot
f2 = bk.figure(width=800, height=720, x_range=[lonmin, lonmax], y_range=[latmin, latmax])
f2.image(image=[SST_anomaly], x=[lonmin], y=[latmin], dw=[lonmax-lonmin], dh=[latmax-latmin], color_mapper=color_mapper)

clabel = f"OSTIA level-4 foundation temperature anomaly ({nrt_year}) for {day} {formatted_month} vs {start_year} to {end_year} climatology"

color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, border_line_color=None, location=(0,0), title=clabel)
f2.add_layout(color_bar, 'right')

bk.show(f2)

<div class="alert alert-info" role="alert">

## 4. <a id='section4'></a>Calculating and adding fronts
[Back to top](#TOC_TOP)
    
</div>

https://github.com/CoLAB-ATLANTIC/JUNO

In [17]:
%%capture
fronts_x, fronts_y = CayulaCornillon_xarray.CCA_SIED(SST_data)

In [18]:
f1.scatter(fronts_x, fronts_y, color="black", alpha=0.5, size=1)
bk.show(f1)

<hr>
<a href="https://github.com/wekeo/GHRSST24" target="_blank">View on GitHub</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>